In [ ]:
# Code of movie recommendations system and it`s reviews and all the details of the movie`s 

In [34]:
from nltk.corpus import movie_reviews
movie_reviews.categories()
movie_reviews.fileids()
len(movie_reviews.fileids())
print(movie_reviews.words(movie_reviews.fileids()[5]))

['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]


In [35]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid), category))
documents[0:5]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')]

In [36]:
import random 
random.shuffle(documents)
documents[0:5]

[(['overblown', 'remake', 'of', 'the', '1963', 'robert', ...], 'neg'),
 (['stallone', 'attempts', 'to', "'", 'act', "'", 'in', ...], 'neg'),
 (['"', 'oh', 'my', 'god', ',', 'i', 'sounded', 'just', ...], 'pos'),
 (['a', 'remake', 'of', 'the', '1978', 'warren', ...], 'neg'),
 (['we', 'share', 'the', 'descent', 'into', 'darkness', ...], 'pos')]

In [37]:
from nltk.corpus import wordnet
def get_simple_pos(tag): 
    if tag.starswith('J'): 
        return wordnet.ADJ 
    elif tag.startswith('V'): 
        return wordnet.VERB
    elif tag.startswith('N'): 
        return wordnet.NOUN
    elif tag.startswith('R'): 
        return wordnet.ADV 
    else: 
        return wordnet.NOUN

In [38]:
from nltk import pos_tag 
w = "better" 
pos_tag([w])

[('better', 'RBR')]

In [39]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
import string

# Set stopwords and punctuation
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)  # Corrected attribute
stops.update(punctuations)

lemmatizer = WordNetLemmatizer()

# Function to map POS tags to simpler tags for lemmatizer
def get_simple_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun

# Function to clean review text
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:  # Check if word is not in stopwords or punctuation
            pos = pos_tag([w])  # Correct usage of pos_tag
            clean_word = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

# Example document structure
# Make sure `documents` is tokenized properly
documents = [
    (["This", "is", "a", "sample", "document"], "category1"),
    (["Another", "example", "text"], "category2"),
]

# Clean reviews
documents = [(clean_review(document), category) for document, category in documents]

print(documents)

training_documents = documents[0:1500]
testing_documents = documents[1500:]

a = [1, 2]
b = [3, 4]
a += b 
a
all_words = []
for doc in training_documents: 
    all_words += doc[0]

print(all_words)

[(['sample', 'document'], 'category1'), (['another', 'example', 'text'], 'category2')]
['sample', 'document', 'another', 'example', 'text']


In [40]:
import nltk 
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]
features
documents[0]

(['sample', 'document'], 'category1')

In [41]:
def get_feature_dict(words): 
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set 
    return current_features 


In [42]:
output = get_feature_dict(training_documents[0][0])
output
training_documents[0][0]

['sample', 'document']

In [44]:
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
test_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
training_data[0]

({'sample': True,
  'document': True,
  'another': False,
  'example': False,
  'text': False},
 'category1')